In [1]:
import numpy as np
import requests, gzip, os, hashlib
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
#fetch data
path='/Users/frankiepinnola/dev/fun/minigrad/examples/data'
def fetch(url):
    fp = os.path.join(path, hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp):
        with open(fp, "rb") as f:
            data = f.read()
    else:
        with open(fp, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(gzip.decompress(data), dtype=np.uint8).copy()

X = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28*28))
Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

In [3]:
print(type(X))
print(X[0].shape)

<class 'numpy.ndarray'>
(28, 28)


In [4]:
X_train = np.array([x.reshape(784,1) for x in X])
X_test = np.array([x.reshape(784,1) for x in X_test])
X_test[0].shape

(784, 1)

In [5]:
from minigrad.engine import Tensor
from minigrad.nn import TMLP

In [6]:
model = TMLP(784, [264,264,1])
print(model)
print(len(model.ind_parameters()))

276939


In [7]:
Y_train = [Tensor(np.array([[v]])) for v in Y]
Y_train[:10]

[Tensor(data=[[5]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[0]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[4]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[1]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[9]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[2]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[1]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[3]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[1]] shape=(1, 1) grad=[[0.]] _op=),
 Tensor(data=[[4]] shape=(1, 1) grad=[[0.]] _op=)]

In [8]:

for _ in range(1000):
    ypred = [model(x) for x in X_train[:2000]]
    loss = sum([(yout - ygt)**2 for ygt, yout in zip(Y_train[:2000],ypred)])
    print(loss)
    
    # zero grad
    model.zero_grad()
    
    
    loss.backward()
    
    for p in model.parameters():
#         print(p.grad)
        p.data -= 0.0000001 * p.grad
    


Tensor(data=[[3.89117439e+12]] shape=(1, 1) grad=[[0.]] _op=+)
Tensor(data=[[1.24542877e+43]] shape=(1, 1) grad=[[0.]] _op=+)
Tensor(data=[[1.82501038e+36]] shape=(1, 1) grad=[[0.]] _op=+)
Tensor(data=[[1.82355066e+36]] shape=(1, 1) grad=[[0.]] _op=+)
Tensor(data=[[1.82209211e+36]] shape=(1, 1) grad=[[0.]] _op=+)
Tensor(data=[[1.82063473e+36]] shape=(1, 1) grad=[[0.]] _op=+)


KeyboardInterrupt: 